In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config('config.json')
print(ws.name, ws.location, ws.resource_group, sep='\t')

In [ ]:
from azureml.core import Dataset

tveer_components = os.path.join(os.getcwd(), 'data/tveer/components')
os.makedirs(tveer_components, exist_ok=True)


In [ ]:
dataset_tveer = Dataset.get_by_name(ws, name='components')

In [ ]:
# mount dataset onto the mounted_path of a Linux-based compute
mnt_ctx_tveer = dataset_tveer.mount(tveer_components)

mnt_ctx_tveer.start()

In [ ]:
component_names = os.listdir("data/tveer/components")

In [ ]:
component_names

In [ ]:
import math
def print_multiple_images(*images, **kwargs):
    amount_of_images = len(images)
    rows = math.ceil(len(images) / 5)
    cols = 5 if len(images) > 5 else len(images)
    fig, axes = plt.subplots(rows, cols, figsize=(24, 3 * rows))
    axes_1d = axes.ravel() if amount_of_images > 1 else [axes]
    for i in range(len(axes_1d)):
        try:
            axes_1d[i].imshow(images[i], **kwargs)
            axes_1d[i].axis('off')    
            axes_1d[i].set_title(images[i].shape, fontdict={'fontsize': 20})
        except IndexError:
            axes_1d[i].axis('off')
    plt.show()

In [ ]:
from glob import glob
all_components = glob('data/tveer/components/')

In [ ]:
import json

In [ ]:
all_components = []
all_labels = []
all_sizes = []
for comp in os.listdir('data/tveer/components'):
    for img_uri in glob(os.path.join('data/tveer/components', comp) + "/*.png"):
        try:
            size = img_uri.split(".png")[-2]
            with open(size + '--size.json', "r") as f:
                all_sizes.append(json.load(f))
                img = plt.imread(img_uri)[:,:,:3]
                all_components.append(img)
                all_labels.append(comp)
        except FileNotFoundError:
            pass

In [ ]:
all_labels_np = np.array(all_labels)
all_components_np = np.array(all_components)
all_sizes_np = np.array(all_sizes)

In [ ]:
print_multiple_images(*all_components_np[:10])
print(all_labels_np[:10])
print(all_sizes_np[:10])

## Train and test splitted data

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sorted(all_labels_np))

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

import math

training_indices = []
test_indices = []
for obj in range(0, onehot_encoded.shape[1]):
    obj_indices = np.where(onehot_encoded[:,obj] == 1)[0]
    np.random.shuffle(obj_indices)
    training_samples = math.floor(0.7 * len(obj_indices))
    training_indices.extend(obj_indices[:training_samples])
    test_indices.extend(obj_indices[training_samples:])

print(f"{len(training_indices)} training indices")
print(f"{len(test_indices)} test indices")

In [ ]:
X_train_conv = all_components_np[training_indices]
X_train_values = all_sizes_np[training_indices]
X_test_conv = all_components_np[test_indices]
X_test_values = all_sizes_np[test_indices]

y_train = onehot_encoded[training_indices]
y_test = onehot_encoded[test_indices]


In [ ]:
train_test_data = os.path.join(os.getcwd(), 'data/tveer/train_test_data')
os.makedirs(train_test_data, exist_ok=True)

In [ ]:
np.save(f"{train_test_data}/X_train_conv.npy", X_train_conv)
np.save(f"{train_test_data}/X_train_values.npy", X_train_values)
np.save(f"{train_test_data}/X_test_conv.npy", X_test_conv)
np.save(f"{train_test_data}/X_test_values.npy", X_test_values)
np.save(f"{train_test_data}/y_train.npy", y_train)
np.save(f"{train_test_data}/y_test.npy", y_test)
np.save(f"{train_test_data}/component_names.npy", np.asarray(component_names))

In [ ]:
from azureml.core import Datastore

In [ ]:
datastore = Datastore(ws)

## Generate an Augmented dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_generator = ImageDataGenerator(rotation_range=360)


In [ ]:
time_to_repeat_generator = 5
image_generator = train_generator.flow([np.repeat(X_train_conv, time_to_repeat_generator, 0), np.repeat(X_train_values, time_to_repeat_generator, 0)], np.repeat(y_train, time_to_repeat_generator, 0), batch_size = 32)

Because we want to keep hold of the data that we trained with, we are going to save a few of this data to a new datastore.

In [ ]:
tveer_generator_path = os.path.join(os.getcwd(), 'data/tveer/generator')
os.makedirs(tveer_generator_path, exist_ok=True)

In [ ]:
from typing import Generator
for i in range(20):
    generated_data, generated_labels = image_generator.next()
    for j in range(generated_data[1].shape[0]):
        plt.imsave(f"{tveer_generator_path}/{component_names[np.argmax(generated_labels[j])]}-{i}-{j}.png", generated_data[0][j])
        with open(f"{tveer_generator_path}/{component_names[np.argmax(generated_labels[j])]}-{i}-{j}--size.json", "w") as f:
            json.dump(generated_data[1][j], f)

In [ ]:
datastore.upload(src_dir='data/tveer/generator', target_path='generated')

In [ ]:
generator_data = Dataset.File.from_files(
    [
        (datastore, 'generated')
    ],
    validate=False)

In [ ]:
generator_dataset = generator_data.register(
    workspace=ws,
    name="generator_data",
    description="Components of the 't Veer dataset which have been generated and slightly augmented with rotations.",
    create_new_version=True)

In [ ]:
from glob import glob
import json

In [ ]:
all_generated_images = []
all_generated_sizes = []
all_generated_labels = []
for img in sorted(glob(f"{tveer_generator_path}/*.png")):
    size_name = img.replace('.png', '--size.json')
    all_generated_images.append(plt.imread(img)[:,:,:3] / 255)
    all_generated_labels.append(img.split('-')[-3].split('/')[-1]) # Get the Object name (data/tveer/generator/Object 1-0-1.png --> Object 1)
    with open(size_name, 'r') as f:
        all_generated_sizes.append(json.load(f))

In [ ]:
all_generated_images_np = np.asarray(all_generated_images)
all_generated_sizes_np = np.asarray(all_generated_sizes)
all_generated_labels = np.asarray(all_generated_labels)

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(all_generated_labels)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
np.save(f"data/tveer/train_test_data/X_train_conv_generated.npy", all_generated_images_np)
np.save(f"data/tveer/train_test_data/X_train_values_generated.npy", all_generated_sizes_np)
np.save(f"data/tveer/train_test_data/y_train_generated.npy", onehot_encoded)

In [ ]:
datastore.upload(src_dir='data/tveer/train_test_data', target_path='train_test_data')

In [ ]:
train_test_dataset_files = Dataset.File.from_files(
    [
        (datastore, 'train_test_data')
    ],
    validate=False)

In [ ]:
train_test_dataset = train_test_dataset_files.register(
    workspace=ws,
    name="train_test_dataset",
    description="A part of the components that have been processed and split in training and testing sets for an AI model.",
    create_new_version=True)